<a href="https://colab.research.google.com/github/bikash-bhoi/eva4/blob/master/Session12/Session_12_TinyImgNet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Clone Github Directory to import packages
%%bash
mkdir temp
git clone https://github.com/bikash-bhoi/eva4.git temp
if [ ! -d ./models ]; then
    mkdir models
fi
if [ ! -d ./utils ]; then
    mkdir utils
fi
cp -r temp/models/* ./models
cp -r temp/utils/* ./utils
rm -rf temp

Cloning into 'temp'...


In [0]:
#!pip install pytorch-gradcam
!pip install albumentations
!pip install livelossplot
#!pip install --no-cache-dir git+https://github.com/NVIDIA/apex

In [0]:
#from utils.download_load import download_load
#from utils.display_imgs import display_imgs
from utils.train_test import train , test, predict, get_misclassified
from utils.disp_summary import disp_summary
#from utils.gradcam import grad_cam, gradcam_plot
from models.resnet import ResNet18,ResNet
from utils.range_test import lr_range_test
from utils.train_model import train_model
#from utils.zig_zag_plot import zigzag_plot

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchsummary
from torchsummary import summary
import torch,os
import torch.utils.data as data
from torch.utils.data import Dataset
import random
from skimage import io

In [0]:
!rm -rf /content/tiny-imagenet-200

In [4]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
#!rm -rf /content/tiny-imagenet-200
!unzip -q "/content/gdrive/My Drive/tiny-imagenet-200.zip"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

data_dir = '/content/tiny-imagenet-200/'
classes = [line.strip() for line in open( data_dir + 'wnids.txt', 'r')]
class_dict = {x:i for i, x in enumerate(classes)}
all_c = {line.split('\t')[0] : line.split('\t')[1].strip() for line in open( data_dir + 'words.txt', 'r')}
class_names = [all_c[x] for x in classes]

imgs = []
lbls = []

for v, c in enumerate(classes):
	imgs += [f'{data_dir}train/{c}/images/{c}_{i}.JPEG' for i in range(500)]
	lbls += [v for i in range(500)]

for line in open( data_dir + 'val/val_annotations.txt'):
	img_name, class_id = line.split('\t')[:2]
	imgs.append(f'{data_dir}val/images/{img_name}')
	lbls.append(class_dict[class_id])

dataset = list(zip(imgs, lbls))
random.shuffle(dataset)


class TinyImagenetReadData(Dataset):

    def __init__(self, data, transform=None):
        self.transform = transform
        self.images, self.labels = zip(*data)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = io.imread(self.images[idx], as_gray=False, pilmode="RGB")

        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

In [0]:
import albumentations
from albumentations import *
from albumentations import Compose
from albumentations.pytorch import ToTensor
import numpy as np

means = (0.48043839, 0.44820218, 0.39760034)
stds = (0.27698959, 0.26908774, 0.28216029)
class TrainAlbumentation():
	def __init__(self):
		self.train_trans = Compose([
							A.PadIfNeeded(min_height=80, min_width=80, border_mode=cv2.BORDER_CONSTANT, value=np.array(means)*255),
							A.RandomCrop(64,64, always_apply=True),
							A.Rotate((-25.0, 25.0)),
							A.HorizontalFlip(),
							A.RGBShift(r_shift_limit=40, g_shift_limit=40, b_shift_limit=40, p=0.5),
							A.Normalize(mean=means, std=stds),
							A.Cutout(num_holes=4, max_h_size=24, max_w_size=24) ,
							ToTensor()
							])

	def __call__(self, im):
		im = np.array(im)
		im = self.train_trans(image = im)['image']
		return im
 
class TestAlbumentation():
	def __init__(self):
		self.train_trans = Compose([A.Normalize(mean=means, std=stds)], ToTensor())

	def __call__(self, im):
		im = np.array(im)
		im = self.train_trans(image = im)['image']
		return im

In [0]:
train.transform

In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import OneCycleLR
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import torchvision
import torchsummary
from torchsummary import summary
from utils.data_albument import TrainAlbumentation,TestAlbumentation


train_bs = 512
test_bs = 512

use_cuda = torch.cuda.is_available()

cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)
SEED=1
# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)
transform_train = TrainAlbumentation()
transform_test = TestAlbumentation()
#transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#trainoader_args = dict(shuffle=True, batch_size=train_bs, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
#testloader_args = dict(shuffle=False, batch_size=test_bs, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

trainlen = int(len(dataset)*0.7)
train = TinyImagenetReadData(dataset[:trainlen], transform=transform_train)
test = TinyImagenetReadData(dataset[trainlen:], transform=transform_test)




CUDA Available? True


In [109]:
train

In [0]:
#Do not use
import torch

class DataLoader:
  def __init__(self, shuffle=True, batch_size=128, seed=1):
    cuda = torch.cuda.is_available()
    
    if cuda:
      torch.cuda.manual_seed(seed)

    self.dataloader_args = dict(shuffle=shuffle, batch_size=batch_size, num_workers=1, pin_memory=True) if cuda else dict(shuffle=shuffle, batch_size=batch_size)

  def load(self, data):
    return torch.utils.data.DataLoader(data, **self.dataloader_args)

In [0]:
dataloader = DataLoader(batch_size=512, shuffle=True)

# train dataloader
train_loader = dataloader.load(train)

# test dataloader
test_loader = dataloader.load(test)

In [8]:
next(iter(train_loader))[0].shape

torch.Size([512, 3, 32, 32])

In [0]:
#import torchvision.models as models
#model = models.resnet18()
#Finetune Final few layers to adjust for tiny imagenet input
#model.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
#model.maxpool = nn.Sequential()
#model.avgpool = nn.AdaptiveAvgPool2d(1)
#model.fc.out_features = 200

model=ResNet18(200)
#model.avgpool = nn.AdaptiveAvgPool2d(1)

use_cuda= torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')
model=model.to(device)

In [10]:
disp_summary(model,[3,64,64])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
         Dropout2d-3           [-1, 64, 64, 64]               0
            Conv2d-4           [-1, 64, 64, 64]          36,864
       BatchNorm2d-5           [-1, 64, 64, 64]             128
         Dropout2d-6           [-1, 64, 64, 64]               0
            Conv2d-7           [-1, 64, 64, 64]          36,864
       BatchNorm2d-8           [-1, 64, 64, 64]             128
         Dropout2d-9           [-1, 64, 64, 64]               0
       BasicBlock-10           [-1, 64, 64, 64]               0
           Conv2d-11           [-1, 64, 64, 64]          36,864
      BatchNorm2d-12           [-1, 64, 64, 64]             128
        Dropout2d-13           [-1, 64, 64, 64]               0
           Conv2d-14           [-1, 64,

/content/models/resnet.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(out)


In [0]:
lrs = [j*(10**i) for i in range(-3,-1) for j in range(1,11)]
lr_range_test(lrs,model,device,train_loader, test_loader)


In [12]:

optimizer = optim.SGD(model.parameters(), lr=0.01,  momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr=0.03,  total_steps=50,pct_start=0.3, final_div_factor=1, div_factor=10)
lrs=[]

for epoch in range(1, 50):
    curr_lr=optimizer.param_groups[0]['lr']
    lrs.append(curr_lr)
    print(f'Epoch: {epoch} Learning_Rate {curr_lr}')
    train_acc1 = train(model, device, train_loader, optimizer, epoch)
    test_acc1 = test(model, device, test_loader)
    print('Test acc:', test_acc1)
    scheduler.step()


Epoch: 1 Learning_Rate 0.002999999999999999


TypeError: ignored

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,  momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr=0.03,  total_steps=50,pct_start=0.3, final_div_factor=1, div_factor=10)
save_path = '/content/gdrive/My Drive/Colab Notebooks/eva4_s12/'
train_model(save_path,model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs=50, scheduler=scheduler)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/50
----------


  0%|          | 0/151 [00:00<?, ?it/s]

Train Loss: -11.1578 Acc: 0.0101
Val Loss: -38.4898 Acc: 0.0000

Epoch 2/50
----------


  0%|          | 0/151 [00:00<?, ?it/s]

...

Train Loss: -43091.1595 Acc: 0.0112
Val Loss: -166963.8848 Acc: 0.0000

Epoch 5/50
----------


  0%|          | 0/151 [00:00<?, ?it/s]

Train Loss: -2621269.2963 Acc: 0.0074
Val Loss: -8614144.5176 Acc: 0.0000

Epoch 6/50
----------


Loss=-671684096.0 Batch_id=127 :  85%|████████▍ | 128/151 [02:21<00:20,  1.14it/s]

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
PATH = '/content/gdrive/My Drive/Colab Notebooks/eva4_s9/assignment11_ResNetcust_model.h5'
!touch PATH
torch.save(model.state_dict(), PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
